# Deep Graph Library(DGL) 基础

Deep Graph Library (DGL)是一个简单通用的图深度学习的库，可以使用PyTorch，TensorFlow和MXNet作为后端框架。利用DGL，可以轻松编写和训练图神经网络，并用于与结构化数据相关的应用。DGL支持许多与图数据相关的功能，比如异质图、点云、小批量加载器和各种常用基准数据集等等。

## DGL的安装

现在DGL支持多种安装方法，见官网https://www.dgl.ai/pages/start.html 。我们推荐使用pip来安装DGL。

注意：正确选择对应的计算机系统、CUDA或者CPU版本。

`pip install dgl -f https://data.dgl.ai/wheels/repo.html`

建议安装DGL最新版，当前是0.9版本，不过上面的命令会安装最新的版本；PyTorch建议1.10.0或更高版本。

`pip install torch==1.10.0`

## DGL图的基本概念和操作

实际业务中要处理的数据绝大部分是异构图。这里我们用下面这张图的异构图来作为例子演示如何构建异构的DGL图。

<img src='heterogeneous_graph.png' width=40%>

该图中包含了公司和产品这两种点，以及制造、下游、供应，一共3种关系。

下面，我们将演示如何从数据中构建异质图。首先，我们从csv文件中读取数据，包括公司（node）信息和产品（node）信息，以及3种边（制造、下游、供应）的信息。

In [2]:
import pandas as pd
import torch as th
import dgl
import numpy as np

df_company     = pd.read_csv('raw_data/companies.csv')
df_product     = pd.read_csv('raw_data/products.csv')
df_produce     = pd.read_csv('raw_data/comp_produce_prod.csv')
df_downstream  = pd.read_csv('raw_data/prod_downstream_prod.csv')
df_supply      = pd.read_csv('raw_data/comp_supply_comp.csv')
print(df_company.keys())
print(df_product.keys())

/home/studio-lab-user/.conda/envs/dgl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Index(['CID', 'size'], dtype='object')
Index(['PID', 'price'], dtype='object')


In [4]:
df_company.head()

,CID,size
0,comp_1,S
1,comp_2,M
2,comp_3,M


### 将节点编号从0重排

在DGL里，我们一般需要对对Node ID从0开始重新排序以满足DGL数据存储的特点。注意这里，公司和产品都从0开始设置ID。

In [6]:
company_mapping = {idx: i for i, idx in enumerate(df_company['CID'].unique())}
num_companies = len(company_mapping)

product_mapping = {idx: i for i, idx in enumerate(df_product['PID'].unique())}
num_products = len(product_mapping)